# Import all Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

2024-01-06 19:01:48.905853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Read the Dataset

In [2]:
df = pd.read_csv("../DATASETS/btc_15m.csv")

In [3]:
df.head()

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13556.15,123.616013
1,2018-01-01 05:45:00,13533.75,13550.87,13402.00,13521.12,98.136430
2,2018-01-01 06:00:00,13500.00,13545.37,13450.00,13470.41,79.904037
3,2018-01-01 06:15:00,13494.65,13690.87,13450.00,13529.01,141.699719
4,2018-01-01 06:30:00,13528.99,13571.74,13402.28,13445.63,72.537533


# Split into Training & Testing Set

In [4]:
dataframe_training, dataframe_testing, y_train, y_test = train_test_split(
    df, df, test_size=0.2, random_state=42
)

# Pre-processing of Training Set

In [5]:
training_set = dataframe_training.iloc[:,1:2].values

In [6]:
sc = MinMaxScaler()
training_set_scaled = sc.fit_transform(training_set)

# Pre-processing of Testing Set

In [16]:
testing_set = dataframe_testing.iloc[:,1:2].values

In [30]:
dataframe_training.head()

,datetime,open,high,low,close,volume
49821,2019-06-07 14:15:00,7958.36,7967.71,7942.91,7954.27,306.504543
64722,2019-11-10 04:30:00,8796.05,8808.42,8790.12,8806.51,150.454382
74911,2020-02-24 18:45:00,9783.04,9811.10,9780.42,9792.54,483.119116
2787,2018-01-30 08:00:00,11040.00,11094.83,11022.01,11032.00,109.619743
24374,2018-09-14 10:00:00,6550.37,6570.00,6539.15,6541.98,228.680587


In [32]:
training_set

array([[ 7958.36],
       [ 8796.05],
       [ 9783.04],
       ...,
       [23722.62],
       [57318.41],
       [36344.84]])

# Number of timesteps

In [7]:
timesteps = 60 # OPTIMISE HYPER-PARAMETER?

In [8]:
X_train = []
Y_train = []

for i in range(timesteps, len(training_set_scaled)):
    #APPEND INTO X_TRAIN: ROWS, COLUMN
    X_train.append(training_set_scaled[i-timesteps:i, 0]) #MEMORY
    Y_train.append(training_set_scaled[i, 0]) #PREDICTIONS

# Convert to Numpy Array

In [18]:
X_train, Y_train = np.array(X_train), np.array(Y_train)

# Reshape to add new dimension

In [33]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Construct the RNN

In [24]:
regressor = Sequential()

In [25]:
regressor.add(LSTM(
    units = 50,
    return_sequences=True, 
    input_shape = (X_train.shape[1], 1)
    ))

regressor.add(Dropout(
    rate = 0.2 
))

regressor.add(LSTM(
    units = 50,  
    return_sequences=True,
    ))

regressor.add(Dropout(
    rate = 0.2 
  ))

regressor.add(LSTM(
    units = 50,
    return_sequences=True,
    ))

regressor.add(Dropout(
    rate = 0.2 
  ))


regressor.add(LSTM(
    units = 50, 
    return_sequences=False,
    ))

regressor.add(Dropout(
    rate = 0.2
  ))


#FINAL - OUTPUT LAYER
regressor.add(Dense(
    units = 1
    ))

2024-01-06 18:50:05.424285: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-06 18:50:05.426329: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-06 18:50:05.427743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

# Compile RNN

In [26]:
regressor.compile(
    optimizer = "adam", #ALWAYS A SAFE CHOICE; RNN, ANN
    loss  = "mean_squared_error" #MSE FOR REGRESSION
    )

# Fit RNN

In [27]:
regressor.fit(
    X_train,
    Y_train,
    epochs = 100,
    batch_size =  32
    )


Epoch 1/100


2024-01-06 18:51:47.078791: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-06 18:51:47.080554: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-06 18:51:47.081794: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "/Users/kpandey/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/kpandey/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/Users